In [ ]:
# Setting seeds for reproducibility
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import ADASYN, SMOTE
import pandas as pd
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
SETTING = 3

In [ ]:
data_file_path = "../../Features/PCFs/files_for_ml/protein_props.json"
druggable_proteins_file_path = "../../DrugBank/druggable_proteins.txt"
approved_druggable_proteins_file_path = "../../DrugBank/approved_druggable_proteins.txt"

with open(data_file_path, 'r') as f:
    protein_data = json.load(f)

print("Total number of uniprot human verified proteins:", len(protein_data))

# Extracting list of druggable and approved druggable proteins
with open(druggable_proteins_file_path, 'r') as f:
    druggable_proteins = f.read().splitlines()

with open(approved_druggable_proteins_file_path, 'r') as f:
    approved_druggable_proteins = f.read().splitlines()

print("Number of druggable proteins:", len(druggable_proteins))
print("Number of approved druggable proteins:", len(approved_druggable_proteins))


# Fetching feature data for all proteins
properties = (pd.read_json("../../Features/PCFs/files_for_ml/protein_props.json")).transpose()
is_druggable = [1 if i in druggable_proteins else 0 for i in properties.index]
is_approved_druggable = [1 if i in approved_druggable_proteins else 0 for i in properties.index]

properties["is_druggable"] = is_druggable
properties["is_approved_druggable"] = is_approved_druggable

PCP_properties = properties.copy()
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
amino_acid_percent = {i:[] for i in amino_acids}
for i in PCP_properties['Amino Acid Percent']:
  for aa in amino_acids:
    amino_acid_percent[aa].append(i[aa])
for aa in amino_acids:
  PCP_properties = pd.concat([PCP_properties, pd.Series(amino_acid_percent[aa], index = PCP_properties.index, name = f"Amino Acid Percent {aa}")], axis = 1)

PCP_properties[f"Molar Extinction Coefficient 1"] = pd.Series([x[0] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)
PCP_properties[f"Molar Extinction Coefficient 2"] = pd.Series([x[1] for x in PCP_properties['Molar Extinction Coefficient']], index = PCP_properties.index)

PCP_properties[f"Secondary Structure helix"] = pd.Series([x[0] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure turn"] = pd.Series([x[1] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)
PCP_properties[f"Secondary Structure sheet"] = pd.Series([x[2] for x in PCP_properties['Secondary Structure']], index = PCP_properties.index)

PCP_properties.drop(columns = ['Amino Acid Count','Amino Acid Percent',"Molar Extinction Coefficient","Flexibility","Secondary Structure",'Sequence'], inplace = True)
PCP_properties['Sequence Length'] = PCP_properties['Sequence Length'].astype(int)
PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']] = PCP_properties[['Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7']].astype(float)

with open("/content/drive/MyDrive/protein_props/features/gdpc_encodings.json", 'r') as file:
    data = json.load(file)
gpdc_encodings = pd.DataFrame(data).transpose()

ppi = pd.read_json("../../Features/PPIs/files_for_ml/ppi.json").transpose()
ppi_network = pd.read_csv("../../Features/PPIs/files_for_ml/ppi_network_properties.csv")
ppi_network.index = ppi_network['Unnamed: 0']
ppi_network.drop(columns = ['Unnamed: 0'], inplace = True)
ppi = pd.concat([ppi, ppi_network], axis = 1)

glycolisation = pd.read_csv("../../Features/PTMs/files_for_ml/glycosylation.csv")
glycolisation.index = glycolisation['Unnamed: 0']
glycolisation.drop(columns = ['Unnamed: 0'], inplace = True)
ptm = pd.read_csv("../../Features/PTMs/files_for_ml/PTM_counts.csv")
ptm.index = ptm["Unnamed: 0"]
ptm.drop(columns = ['Unnamed: 0'], inplace = True)
ptm_counts = pd.concat([ptm, glycolisation], axis = 1)

with open("../../Features/SCL/files_for_ml/subcellular_locations2.json", 'r') as file:
    data = json.load(file)
unique_groups = set()
for entry in data.values():
    if "general" in entry:
        for general_entry in entry["general"]:
            if "group" in general_entry: unique_groups.add(general_entry["group"])

unique_groups_list = list(unique_groups)

rows = []
for protein_id in PCP_properties.index:
    row = {group: 0 for group in unique_groups_list}
    if protein_id in data:
        for entry in data[protein_id].get("general", []):
            if "group" in entry and entry["group"] in unique_groups:
                row[entry["group"]] = 1
    row["protein_id"] = protein_id
    rows.append(row)

subcellular_data = pd.DataFrame(rows).set_index("protein_id")

domains = pd.read_csv("../../Features/Domains/files_for_ml/data_top20.csv")
domains.index = domains['Unnamed: 0']
domains.drop(columns = ['Unnamed: 0'], inplace = True)

flexibility = pd.read_csv("../../Features/PCFs/files_for_ml/flexibility_properties.csv")
flexibility.index = flexibility['Unnamed: 0']
flexibility.drop(columns = ['Unnamed: 0'], inplace = True)

latent_data = pd.read_csv("../../Features/Latents/files_for_ml/latent_values.csv").transpose()
latent_data.columns = [f"Latent_Value_{i+1}" for i in latent_data.columns]
final_data = pd.concat([PCP_properties,gpdc_encodings, ptm_counts, ppi, subcellular_data, domains, flexibility, latent_data], axis = 1).dropna()
features_list = final_data.columns
features_list = features_list.drop(['is_druggable','is_approved_druggable'])
features_list = list(features_list)
print(features_list)
print(len(features_list))


Total number of uniprot human verified proteins: 20434
Number of druggable proteins: 3345
Number of approved druggable proteins: 2652
['Sequence Length', 'Molecular Weight', 'GRAVY', 'Isoelectric Point', 'Instability Index', 'Aromaticity', 'Charge at 7', 'Amino Acid Percent A', 'Amino Acid Percent C', 'Amino Acid Percent D', 'Amino Acid Percent E', 'Amino Acid Percent F', 'Amino Acid Percent G', 'Amino Acid Percent H', 'Amino Acid Percent I', 'Amino Acid Percent K', 'Amino Acid Percent L', 'Amino Acid Percent M', 'Amino Acid Percent N', 'Amino Acid Percent P', 'Amino Acid Percent Q', 'Amino Acid Percent R', 'Amino Acid Percent S', 'Amino Acid Percent T', 'Amino Acid Percent V', 'Amino Acid Percent W', 'Amino Acid Percent Y', 'Molar Extinction Coefficient 1', 'Molar Extinction Coefficient 2', 'Secondary Structure helix', 'Secondary Structure turn', 'Secondary Structure sheet', 'aliphatic_aliphatic', 'aliphatic_positive', 'aliphatic_negative', 'aliphatic_uncharged', 'aliphatic_aromatic',

In [ ]:
# Train Test Splitting
def data_splitting(x_sample, y_sample, mode="default", scaler="none"):
  druggable_indices = (y_sample == 1)  # Assuming 1 represents druggable
  non_druggable_indices = (y_sample == 0)  # Assuming 0 represents non-druggable

  druggable_X = x_sample[druggable_indices]
  druggable_y = y_sample[druggable_indices]

  non_druggable_X = x_sample[non_druggable_indices]
  non_druggable_y = y_sample[non_druggable_indices]

  class_size = 600
  druggable_X_remaining, druggable_X_test, druggable_y_remaining, druggable_y_test = train_test_split(druggable_X, druggable_y, test_size=class_size, random_state=123)
  non_druggable_X_remaining, non_druggable_X_test, non_druggable_y_remaining, non_druggable_y_test = train_test_split(non_druggable_X, non_druggable_y, test_size= class_size, random_state=123)

  X_test = pd.concat((druggable_X_test, non_druggable_X_test))
  y_test = pd.concat((druggable_y_test, non_druggable_y_test))
  X_train = pd.concat((druggable_X_remaining, non_druggable_X_remaining))
  y_train = pd.concat((druggable_y_remaining, non_druggable_y_remaining))
  X_train, y_train = shuffle(X_train, y_train, random_state=123)
  if mode == "default":
    pass
  elif mode == "adasyn":
    ada = ADASYN(random_state=42)
    X_train, y_train = ada.fit_resample(X_train, y_train)
  elif mode == "smote":
    smt = SMOTE(random_state=42)
    X_train, y_train = smt.fit_resample(X_train, y_train)

  if scaler == "std":
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "minmax":
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
  elif scaler == "none":
    pass

  return X_train, X_test, y_train, y_test


In [ ]:
# rem-new-data is to extract only those proteins which are either approved druggable or non-druggable
# i.e., it excludes proteins which are non-approved but druggable
new_data = final_data.copy()
new_data['new_column'] = new_data['is_druggable'] + new_data['is_approved_druggable']
rem_new_data = new_data[new_data['new_column'] != 1]
rem_new_data.shape, np.bincount(rem_new_data['new_column'])

((19585, 186), array([16949,     0,  2636]))

In [ ]:
rem_new_data.head()

,Sequence Length,Molecular Weight,GRAVY,Isoelectric Point,Instability Index,Aromaticity,Charge at 7,is_druggable,is_approved_druggable,Amino Acid Percent A,...,Latent_Value_12,Latent_Value_13,Latent_Value_14,Latent_Value_15,Latent_Value_16,Latent_Value_17,Latent_Value_18,Latent_Value_19,Latent_Value_20,new_column
A0A087X1C5,515,57488.0269,-0.016117,8.703506,45.155922,0.085437,5.889114,0,0,0.081553,...,-59.412537,-21.617113,-11.201081,-14.104016,11.058478,-37.962920,-65.891304,20.351254,-20.354603,0
A0A0B4J2F0,54,6313.3024,-0.235185,8.035420,55.518519,0.129630,0.510326,0,0,0.074074,...,-56.975170,-20.523827,-12.358631,-13.939702,11.742297,-35.848106,-62.440926,19.577995,-19.569004,0
A0A0B4J2F2,783,84929.1856,-0.197957,6.813029,59.277803,0.063857,-1.004586,0,0,0.084291,...,-60.115986,-21.257944,-11.745778,-14.150418,11.397512,-38.209408,-65.968510,21.196396,-20.730612,0
A0A0C5B5G6,16,2174.5917,-0.937500,10.266413,77.300625,0.250000,2.500138,0,0,0.000000,...,-72.063614,-25.551811,-13.592831,-17.069069,13.195458,-47.066390,-80.288940,25.196339,-25.211790,0
A0A0K2S4Q6,201,21805.9293,0.107960,5.366988,41.796517,0.074627,-3.378625,0,0,0.054726,...,-64.386490,-23.750786,-13.102545,-15.498997,12.353992,-41.174583,-71.124260,22.808182,-21.874224,0


In [ ]:
if SETTING == 1:
  X_train, X_test, y_train, y_test = data_splitting(final_data[features_list], final_data['is_druggable'])
elif SETTING == 2:
  X_train, X_test, y_train, y_test = data_splitting(final_data[features_list], final_data['is_approved_druggable'])
elif SETTING == 3:
  X_train, X_test, y_train, y_test = data_splitting(rem_new_data[features_list], rem_new_data['is_druggable'])

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18385, 183), (1200, 183), (18385,), (1200,))

In [ ]:
# Distribution of classes in train and test sets
np.bincount(y_train), np.bincount(y_test)

(array([16349,  2036]), array([600, 600]))

### Feature Selection Scores

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Combined data
X_combined = np.concatenate((X_train, X_test))
y_combined = np.concatenate((y_train, y_test))

# Data Shuffling
X_combined, y_combined = shuffle(X_combined, y_combined, random_state=123)
X_combined.shape, y_combined.shape

((19585, 183), (19585,))

### Feature Scores using Entire Train Data on a Single XGB Model

In [ ]:
import xgboost as xgb
xgb_model_fs1 = xgb.XGBClassifier(objective='binary:logistic', random_state=27)
xgb_model_fs1.fit(X_combined, y_combined)

print("Accuracy_Total:", xgb_model_fs1.score(X_combined, y_combined))
print("Accuracy_Druggable:", xgb_model_fs1.score(X_combined[y_combined == 1], y_combined[y_combined == 1]))
print("Accuracy_Non_Druggable", xgb_model_fs1.score(X_combined[y_combined == 0], y_combined[y_combined == 0]))

Accuracy_Total: 0.9926984937452131
Accuracy_Druggable: 0.9465098634294385
Accuracy_Non_Druggable 0.9998819989379905


In [ ]:
fs1_scores = xgb_model_fs1.feature_importances_
np.bincount(y_combined)

array([16949,  2636])

### Feature Scores using Partition Method of Train Data using several XGB models

In [ ]:
X_combined_druggable = X_combined[y_combined == 1]
X_combined_non_druggable = X_combined[y_combined == 0]
number_partitions = int(X_combined_non_druggable.shape[0]/X_combined_druggable.shape[0])
X_combined_non_druggable_partitions = np.array_split(X_combined_non_druggable, number_partitions)

print("Number of partitions:", len(X_combined_non_druggable_partitions))
for partition in X_combined_non_druggable_partitions:
  print(partition.shape)

Number of partitions: 6
(2825, 183)
(2825, 183)
(2825, 183)
(2825, 183)
(2825, 183)
(2824, 183)


In [ ]:
xgb_models = []
training_metrics = {}
for i, partition in enumerate(X_combined_non_druggable_partitions):
  X_train_new = np.concatenate((X_combined_druggable, partition))
  y_train_new = np.concatenate((np.ones(X_combined_druggable.shape[0]), np.zeros(partition.shape[0])))
  X_train_new, y_train_new = shuffle(X_train_new, y_train_new, random_state=123)
  xgb_model_fs2 = xgb.XGBClassifier(objective='binary:logistic', random_state=27)
  xgb_model_fs2.fit(X_train_new, y_train_new)
  xgb_models.append(xgb_model_fs2)

  training_metrics[f"partition_{i}"] = {
      "accuracy_total" : xgb_model_fs2.score(X_train_new, y_train_new),
      "accuracy_druggable": xgb_model_fs2.score(X_train_new[y_train_new == 1], y_train_new[y_train_new == 1]),
      "accuracy_non-druggable": xgb_model_fs2.score(X_train_new[y_train_new == 0], y_train_new[y_train_new == 0])
  }

In [ ]:
test_metrics = {}
for i in range(number_partitions):
  model = xgb_models[i]
  remaining_non_druggable_partitions = []
  for j in range(number_partitions):
    if j != i:
      remaining_non_druggable_partitions.append(X_combined_non_druggable_partitions[j])
  remaining_non_druggable_partitions = np.concatenate(remaining_non_druggable_partitions)
  test_metrics[f"partition_{i}"] = {
      "accuracy_non_druggable": model.score(remaining_non_druggable_partitions, np.zeros(remaining_non_druggable_partitions.shape[0]))
  }


In [ ]:
training_metrics, test_metrics

({'partition_0': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0},
  'partition_1': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0},
  'partition_2': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0},
  'partition_3': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0},
  'partition_4': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0},
  'partition_5': {'accuracy_total': 1.0,
   'accuracy_druggable': 1.0,
   'accuracy_non-druggable': 1.0}},
 {'partition_0': {'accuracy_non_druggable': 0.7898612291135656},
  'partition_1': {'accuracy_non_druggable': 0.7766921551968281},
  'partition_2': {'accuracy_non_druggable': 0.7904984423676013},
  'partition_3': {'accuracy_non_druggable': 0.7783205890682526},
  'partition_4': {'accuracy_non_druggable': 0.7744265080713679},
  'partition_5': {'accuracy_non_drugga

In [ ]:
def get_dict_form(feature_scores):
  return {feature: score for feature, score in zip(features_list, feature_scores)}

In [ ]:
mean_feature_importances = np.mean([model.feature_importances_ for model in xgb_models], axis=0)

In [ ]:
model_fs_scores = {"All_Data": get_dict_form(xgb_model_fs1.feature_importances_)}
for i in range(number_partitions):
  model_fs_scores[f"Partition_{i+1}"] = get_dict_form(xgb_models[i].feature_importances_)
model_fs_scores["Partition_Average"] = get_dict_form(mean_feature_importances)

In [ ]:
SETTING

3

In [ ]:
# Non Druggable v/s Approved Druggable
df = pd.DataFrame(model_fs_scores)
if SETTING == 3:
  df.to_csv("XGB/partition_avg_feature_scores.csv")

In [ ]:
df.head()

,All_Data,Partition_1,Partition_2,Partition_3,Partition_4,Partition_5,Partition_6,Partition_Average
Sequence Length,0.009233,0.010499,0.010022,0.006765,0.008323,0.006509,0.008435,0.008425
Molecular Weight,0.007722,0.012308,0.003977,0.012389,0.008871,0.012998,0.010288,0.010139
GRAVY,0.004843,0.005122,0.006508,0.008423,0.008382,0.007108,0.014064,0.008268
Isoelectric Point,0.005614,0.007063,0.004046,0.004635,0.005024,0.005251,0.003844,0.004977
Instability Index,0.022150,0.021710,0.021889,0.016820,0.021773,0.017765,0.014454,0.019069


### Experimenting with Increasing Number of Top Features based on Feature Selection scores from Partition Average

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

def evaluate_partition_method_model(features, class_size):
  if SETTING == 1:
    X_test, X_train, y_test, y_train = data_splitting(final_data[features], final_data['is_druggable'], class_size = class_size)
  elif SETTING == 2:
    X_test, X_train, y_test, y_train = data_splitting(final_data[features], final_data['is_approved_druggable'], class_size = class_size)
  elif SETTING == 3:
    X_train, X_test, y_train, y_test = data_splitting(rem_new_data[features], rem_new_data['is_druggable'], class_size = class_size)
  if len(features) == 1: # Printing for the first time
    print(f"X_train shape is {X_train.shape}")
    print(f"X_test shape is {X_test.shape}")
    print(f"y_train distribution is {np.bincount(y_train)}")
    print(f"y_test distribution is {np.bincount(y_test)}")

  X_train_druggable = X_train[y_train==1]
  X_train_non_druggable = X_train[y_train==0]
  X_train_non_druggable_partitions = np.array_split(X_train_non_druggable, round(len(X_train_non_druggable)/len(X_train_druggable)))

  if len(features) == 1: # Printing for the first time
    print(f"Splitting into {len(X_train_non_druggable_partitions)} partitions")
    print("Sizes of partitions")
    for i, partition in enumerate(X_train_non_druggable_partitions):
      print(f"Partition {i}: {len(partition)}")

  xgb_models = []
  for partition in X_train_non_druggable_partitions:
    X_train_combined = np.concatenate((np.array(X_train_druggable), np.array(partition)))
    y_train_combined = np.concatenate((np.ones(len(X_train_druggable)), np.zeros(len(partition))))
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
    xgb_model.fit(X_train_combined, y_train_combined)
    xgb_models.append(xgb_model)

  y_pred_probas = []
  for model in xgb_models:
    y_pred_probas.append(model.predict_proba(X_test)[:, 1])
  average_pred_probas = np.mean(np.array(y_pred_probas), axis=0)
  y_pred = np.round(average_pred_probas)

  return {
      "test_accuracy": accuracy_score(y_test, y_pred),
      "test_accuracy_druggable": accuracy_score(y_test[y_test == 1], y_pred[y_test == 1]),
      "test_accuracy_non_druggable": accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
  }


In [ ]:
if SETTING == 3:
  fs_scores = pd.read_csv("XGB/partition_avg_feature_scores.csv")
fs_scores_all_data = fs_scores["All_Data"].values
fs_scores_partition_avg = fs_scores["Partition_Average"].values

fs_scores_all_data = {feature: score for feature, score in zip(features_list, fs_scores_all_data)}
fs_scores_partition_avg = {feature: score for feature, score in zip(features_list, fs_scores_partition_avg)}

fs_scores_all_data = {k: v for k, v in sorted(fs_scores_all_data.items(), key=lambda item: item[1], reverse=True)}
fs_scores_partition_avg = {k: v for k, v in sorted(fs_scores_partition_avg.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
CLASS_SIZE = 600

metrics_with_fs_scores_partition_avg = {}
for i in range(1, len(features_list)+1):
  features = list(fs_scores_partition_avg.keys())[:i]
  metrics_with_fs_scores_partition_avg[i] = evaluate_partition_method_model(features,CLASS_SIZE)
  print(f"{i} Done with accuracy {metrics_with_fs_scores_partition_avg[i]['test_accuracy']}")


In [ ]:
# df1 = pd.DataFrame(metrics_with_fs_scores_all_data).transpose()
df2 = pd.DataFrame(metrics_with_fs_scores_partition_avg).transpose()

In [ ]:
if SETTING == 3:
    df2.to_csv("XGB/feature_improvement_metrics.csv")

In [ ]:
df2.head()